## Position Estimator With ReID

In [1]:
from src.sensor import Camera

mba_video_camera = Camera(
    h_fov=59.0,
    h_res=1080,
    v_res=720,
)

mba_photo_camera = Camera(
    h_fov=67.0,
    h_res=1290,
    v_res=720,
)

zed_two_camera = Camera(
    h_fov=110.0,
    v_fov=70.0,
)

avg_shoulder_width = 350.0
avg_torso_height = 460.0


In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
from src.point import Point
from src.vision import PositionEstimator, VideoCapture
import mediapipe as mp
from src.vision import VideoCapture
from src.vision import ObjectDetector
from src.vision import plot_obj
from ultralytics import YOLO
from src.vision import ReID
import time

In [3]:
verbose = False
refresh_rate = 1
camera = mba_video_camera
yolo_model = YOLO('yolov8s.pt')
obj_detector = ObjectDetector(yolo_model, camera)
pe = PositionEstimator(
    mp.solutions.pose, 
    camera,
    avg_torso_height, 
    file_path='io/pe.csv',
    queue_size=3,
    kernel_size=3
)


try:
    # Initiate the camera
    # cap = VideoCapture('./Movie on 2023-11-23 at 10.03.mov')
    # cap = VideoCapture('./Movie on 2023-12-11 at 21.01.mov')
    # cap = VideoCapture('./Movie on 2023-12-11 at 21.02.mov')
    cap = VideoCapture(0)
    time.sleep(1)

    # cap.skip_frames(14)
    frame = cap.grab_frame()
    obj_detector.detect(frame)
    host = obj_detector.get_objs_by_name('person', score=0.4).iloc[0]
    reid = ReID(frame, host, 32, 0.80)

    while True:
        start = time.time()
        # cap.skip_frames(5)
        frame = cap.grab_frame()
        if frame is None: break
        
        obj_detector.detect(frame)
        objs = obj_detector.get_objs_by_name('person', score=0.4)
        host, objs = reid.reid(frame, objs)

        clear_output(wait=True)
        fig, ax = plt.subplots() if verbose else (None, None)

        if host is not None:
            bo_frame = PositionEstimator.mask_frame(frame, host)
            result = pe.save_position(bo_frame, 1.15, verbose=verbose)
            if result is not None and verbose:
                print('result')
                pe.plot_result(ax, frame, result, camera)
        else:
            pe.save_blank()

        ax.imshow(frame) if verbose else None
        # ax.imshow(bo_frame) if verbose else None

        if verbose:
            plot_obj(ax, host, 'r') if host is not None else None
            for index, obj in objs.iterrows():
                plot_obj(ax, obj, 'y')
            plt.axis('off')
            plt.show()

        delta = time.time() - start
        # print(f'FPS: {1/delta:.2f}')
        pause = (1 / refresh_rate) - delta
        if pause > 0: time.sleep(pause)
        delta = time.time() - start
        # print(f'FPS: {1/delta:.2f}')

except KeyboardInterrupt:
    cap.release()  # Release the camera resource

cap.release()  # Release the camera resource if break

I0000 00:00:1702636612.208564       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
[ WARN:0@3.910] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
2023-12-15 18:36:52.491 python[47119:1684733] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
